<h1>Kaggle: Bluebook for Bulldozers<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Descripción" data-toc-modified-id="Descripción-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Descripción</a></span></li><li><span><a href="#Carga-de-los-datos" data-toc-modified-id="Carga-de-los-datos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Carga de los datos</a></span></li><li><span><a href="#Análisis-exploratorio" data-toc-modified-id="Análisis-exploratorio-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Análisis exploratorio</a></span></li><li><span><a href="#Preprocesado-de-datos" data-toc-modified-id="Preprocesado-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocesado de datos</a></span><ul class="toc-item"><li><span><a href="#Categorías" data-toc-modified-id="Categorías-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Categorías</a></span></li><li><span><a href="#Fechas" data-toc-modified-id="Fechas-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Fechas</a></span></li><li><span><a href="#Valores-NA,-etc" data-toc-modified-id="Valores-NA,-etc-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Valores NA, etc</a></span></li><li><span><a href="#Subconjuntos-de-entrenamiento-y-validación" data-toc-modified-id="Subconjuntos-de-entrenamiento-y-validación-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Subconjuntos de entrenamiento y validación</a></span></li></ul></li><li><span><a href="#Creación,-entrenamiento-y-evaluación-del-modelo" data-toc-modified-id="Creación,-entrenamiento-y-evaluación-del-modelo-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creación, entrenamiento y evaluación del modelo</a></span></li></ul></div>

> Notebook para jugar con los datos de la competición de Kaggle [bluebook-for-bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers)

## Descripción

"El objetivo del concurso es predecir el precio de venta de una pieza particular de equipo pesado en una subasta, en función de su uso, tipo de equipo y configuración. Los datos provienen de publicaciones de resultados de la subasta e incluyen información sobre el uso y las configuraciones del equipo.

Fast Iron está creando un "libro azul para los bulldozers", con el fin de que los clientes valoren correctamente su flota de cara a una subasta."

* Descargas + Información completa sobre los datasets: https://www.kaggle.com/c/bluebook-for-bulldozers/data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import os

In [3]:
set_plot_sizes(12,14,16)

## Carga de los datos

Primero tendremos que descargar los datos desde la [web de Kaggle](https://www.kaggle.com/c/bluebook-for-bulldozers/data), y los dejamos en el subdirectorio /data.

A continuación volcamos la información del fichero en un dataframe de Pandas. Usaremos la función info para ver sus muestras (filas) y sus variables (columnas). De estas últimas obtendremos también el tipo y el número de valores que no son NaN.

In [4]:
df_raw = pd.read_csv(f'data/Train.csv', low_memory=False, 
                     parse_dates=["saledate"])  # use this for all columns with dates
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                

## Análisis exploratorio

Lo primero que haremos siempre después de cargar los datos será echar un primer vistazo a los mismos, para saber con qué estamos tratando. Creamos una función auxiliar (recubriendo a la función display de Python) para visualizar el contenido de un dataframe con un límite preestablecido de filas y columnas, y vemos una muestra de nuestro dataframe. Usamos T para transponer filas por columnas y visualizar mejor los datos en el notebook:

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        display(df)

display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


Nuestra variable objetivo o valor a predecir se trata de la columna 'SalePrice'. El resto serán variables dependientes.

Echamos también un vistazo a los estadísticos básicos para todas las columnas, lo que nos dará un poco de información para conocer el contenido de las columnas o variables dependientes:

In [6]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En este caso no vamos a hacer más análisis exploratorio, ya que no tiene mucho sentido si no pensamos quitar variables.

Vamos a guardar el dataframe en un fichero para no tener que volver a repetir la carga:

In [7]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [8]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

## Preprocesado de datos

Siempre es importante saber qué tipo de métrica se va a usar en un proyecto para la evaluación del modelo. En este caso Kaggle nos dice que evaluará usando RMSLE (Root Mean Squared Log Error), entre la predicción y el valor real de cada precio (columna SalePrice). Por ello usaremos el logaritmo del precio, y así el error RMSE nos dará directamente el RMLSE:

In [9]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

Para las columnas que contienen las variables dependientes observamos que tenemos una mezcla de datos numéricos, fechas y datos categóricos. 

### Categorías

Para las categorías lo que hacemos es convertir las columnas cuyos valores son strings al tipo category de Pandas. De tal forma que para Pandas ya es como si fueran variables numéricas, puesto que internamente asigna a cada string un código numérico.

Primeramente podemos echar un vistazo al número de valores únicos dentro de cada variable:

In [10]:
unique_counts = pd.DataFrame.from_records([(col, df_raw[col].nunique()) 
                                           for col in df_raw.columns  if df_raw[col].dtype == 'object'],
                                           columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

,Column_Name,Num_Unique
30,Coupler_System,2
19,Blade_Extension,2
22,Engine_Horsepower,2
13,Forks,2
24,Pushblock,2
26,Scarifier,2
16,Stick,2
18,Turbocharged,2
33,Track_Type,2
39,Backhoe_Mounting,2


Usamos la función `train_cats` para convertir todas las columnas del dataframe de tipo object a tipo category:

In [11]:
train_cats(df_raw)

In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null category
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null category
fiBaseModel                 401125 non-null category
fiSecondaryDesc             263934 non-null category
fiModelSeries               56908 non-null category
fiModelDescriptor           71919 non-null category
ProductSize                 190350 non-null category
fiProductClassDesc          401125 non-null category
sta

Además de nuestro objetivo hemos conseguido reducir el tamaño en memoria del dataframe a una cuarta parte del original!

Vemos un ejemplo de variable:

In [13]:
df_raw.state.cat.categories

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Unspecified', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'Washington DC', 'West Virginia', 'Wisconsin',
       'Wyoming'],
      dtype='object')

Para la variable UsageBand observamos que tenemos unas categorías donde podría importar el orden:

In [14]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

Y como el orden es alfabético, esto podría influir de forma negativa a la hora de construir los árboles de decisión, puesto que lo natural es High > Medium > Low. Pero es algo que podemos cambiar:

In [15]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
df_raw.UsageBand.cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

Este último cambio en el mapeo afectará a nuestro conjunto de entrenamiento. Obviamente queremos que el mapeo entre categorías y códigos sea igual para validación y test, para lo que fastai nos proporciona la función `apply_cats`, que ejecutada sobre dichos dataframes, aplicará el mismo mapeo que existía en entrenamiento.

In [16]:
?apply_cats

### Fechas

Para las fechas usamos `add_datepart`, que convertirá una columna de tipo datetime en varias numéricas. Lo aplicamos sobre la columna 'saledate', que ya es un datetime por la forma en que cargamos el CSV inicialmente :)

In [17]:
add_datepart(df_raw, 'saledate')

In [18]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


Vemos cómo se han añadido las columnas: 'saleYear', 'saleMonth', 'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',       'saleIs_month_end', 'saleIs_month_start', 'saleIs_quarter_end', 'saleIs_quarter_start', 'saleIs_year_end', 'saleIs_year_start', 'saleElapsed'.

### Valores NA, etc

Como hemos podido ver en nuestro primer análisis exploratorio, tenemos un montón de NaN, que no podemos pasar al algoritmo Random Forests. Por suerte contamos con la función `proc_df` de fastai, que nos resuelve varias cuestiones:
 * Ajustar el mapeo entre categorías y números (mediante numericalize, que convierte -1 en 0, 0 en 1, etc) y quedarse únicamente con los valores numéricos.
 * Crear dummies (categorías con pocos valores posibles)
 * Manejar los missing values en columnas numéricas (fix_missing). No hace falta en las categóricas porque Pandas ya los tradujo a -1.
 * Separar la variable dependiente del resto

In [19]:
 proc_df??

In [20]:
df, y, nas = proc_df(df_raw, 'SalePrice')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 66 columns):
SalesID                        401125 non-null int64
MachineID                      401125 non-null int64
ModelID                        401125 non-null int64
datasource                     401125 non-null int64
auctioneerID                   401125 non-null float64
YearMade                       401125 non-null int64
MachineHoursCurrentMeter       401125 non-null float64
UsageBand                      401125 non-null int8
fiModelDesc                    401125 non-null int16
fiBaseModel                    401125 non-null int16
fiSecondaryDesc                401125 non-null int16
fiModelSeries                  401125 non-null int8
fiModelDescriptor              401125 non-null int16
ProductSize                    401125 non-null int8
fiProductClassDesc             401125 non-null int8
state                          401125 non-null int8
ProductGroup                   401125 non-

In [21]:
df.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,3,3,1,1,2
fiModelDesc,950,1725,331,3674,4208
fiBaseModel,296,527,110,1375,1529


### Subconjuntos de entrenamiento y validación

Kaggle nos proporciona los siguientes ficheros de datos:
 * Train.csv : datos hasta 2011. Usados para entrenar el modelo
 * Valid.csv : datos de los primeros 4 meses de 2012. Usados para validar el modelo -> Public LeaderBoard
 * Test.csv : datos desde mayo a noviembre de 2012. Usados para determinar el ranking final -> Private LeaderBoard
 
Lo que vamos a hacer nosotros en una competición de Kaggle normalmente es usar Valid.csv como test, y dividir Train.csv entre entrenamiento y validación. El motivo es que Test.csv no se libera hasta la última semana.

In [27]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df) - n_valid

X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

## Creación, entrenamiento y evaluación del modelo

Una vez pre-procesado el dataset, procedemos con el modelo. Primero creamos unas funciones auxiliares para calcular todas las medidas necesarias:

In [23]:
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    # RMSE for training, RMSE for validation, r^2 (accuracy) for training, r^2 for validation, OoB score
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('RMSE for training:   ', res[0])
    print('RMSE for validation: ', res[1])
    print('R^2 for training:    ', res[2])
    print('R^2 for validation:  ', res[3])
    if hasattr(m, 'oob_score_'): print('OoB error:           ', 1 - res[4])

A continuación instanciamos la clase adecuada con el nº de estimadores (árboles de decisión) por defecto, y paralelización de trabajos:

In [24]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 17.3 s
RMSE for training:    0.0904804411213391
RMSE for validation:  0.25066216066832697
R^2 for training:     0.9828902271218335
R^2 for validation:   0.8877915581292751


Parece que nuestro modelo tiene un problema de sobreajuste por la diferencia que observamos entre los resultados de entrenamiento y validación. Introducimos una característica de Random Forests para corroborarlo; el Out of Bag error, que mide el error de predicción en modelos basados en bagging (subimos el nº de estimadores para que sklearn pueda calcularlo sin problemas). Este error se usa en realidad cuando no tenemos datos de validación.

In [25]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 57 s
RMSE for training:    0.0782024356167586
RMSE for validation:  0.2374541506857948
R^2 for training:     0.9872186895165079
R^2 for validation:   0.8993050930671705
OoB error:            0.09112550641946071


Este último error es bastante elevado, lo que confirma el sobreajuste.